In [3]:
import bz2
import json
import numpy as np
import pandas as pd
import spacy
import pickle
from pathlib import Path

from IPython.display import display, HTML

from ressources import config

In [5]:
RAW_DATA_FOLDER = config.RAW_DATA_FOLDER
GENERATED_DATA_FOLDER = config.GENERATED_DATA_FOLDER

In [ ]:
def sanitarization(word_list):
    nlp = spacy.load("en_core_web_sm")
    word_list_san = []   
    for film in word_list:
        doc_film_name = nlp(film)
        tokens = [token.text for token in doc_film_name]
        result = " ".join(tokens)
        word_list_san.append(result)
    return word_list_san

## QUOTEBANK Dataset
Exploring and filtering of quotbank Dataset

In [14]:
QUOTEBANK_FOLDER = RAW_DATA_FOLDER / "QUOTEBANK"
file_list = list(QUOTEBANK_FOLDER.glob('*.json.bz2'))

### List of features

In [15]:
with pd.read_json(file_list[0], lines=True, compression='bz2', chunksize=1) as df_reader:
    for chunk in df_reader:
        df_quotebank = chunk
        break
# column list for Quotebank dataset
print(f"\nColumns quotebank:\n{df_quotebank.columns}")


Columns quotebank:
Index(['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
       'probas', 'urls', 'phase'],
      dtype='object')


### Sample

In [16]:
# Sample for Quotebank dataset
print("\nSample quotebank:\n")
display(df_quotebank)


Sample quotebank:



,quoteID,quotation,speaker,qids,date,numOccurrences,probas,urls,phase
0,2018-06-30-000005,... a minimum of 5.25 trillion (plastic) parti...,Marcus Eriksen,[Q55997400],2018-06-30 07:00:00,3,"[[Marcus Eriksen, 0.6814], [None, 0.3186]]",[http://www.santacruzsentinel.com/environment-...,E


### Number of rows

In [ ]:
# Number of rows for quotebank dataset
chunksize = 10000
for file in file_list:
    n = 0
    print(f"Processing file {file.name}")
    with pd.read_json(file, lines=True, compression='bz2', chunksize=chunksize) as df_reader:
        for chunk in df_reader:
            n += len(chunk)
            print(n, end = "\r")
    
    print(f"{n} rows in {file.name}")

Output:

Processing file quotes-2015.json.bz2</br>
20874338 rows in quotes-2015.json.bz2</br>
Processing file quotes-2016.json.bz2</br>
13862129 rows in quotes-2016.json.bz2</br>
Processing file quotes-2017.json.bz2</br>
26611588 rows in quotes-2017.json.bz2</br>
Processing file quotes-2018.json.bz2</br>
27228451 rows in quotes-2018.json.bz2</br>
Processing file quotes-2019.json.bz2</br>
21763302 rows in quotes-2019.json.bz2</br>

### Filtering out data
We decided to keep only the data that contains some keywords in the quote itself or in the url

In [ ]:
for file in file_list:
    path_to_out = GENERATED_DATA_FOLDER / "QUOTEBANK"
    path_to_out = path_to_out / f"{file.name.split('.', 1)[0]}-cinema.{file.name.split('.', 1)[1]}"
    with bz2.open(file, 'rb') as in_file:
        with bz2.open(path_to_out, 'wb') as out_file:
            for instance in in_file:
                instance = json.loads(instance)
                quote = instance['quotation']
                urls = instance['urls']
                if 'cinema' in quote or 'film' in quote or 'movie' in quote:
                    d_file.write((json.dumps(instance)+'\n').encode('utf-8'))
                elif:
                    for url in urls:
                        if 'cinema' in url or 'film' in url or 'movie' in url:
                            d_file.write((json.dumps(instance)+'\n').encode('utf-8'))
                            break